## THIS NOTEBOOK WILL NOT WORK IN GOOGLE COLAB!!! IT MUST BE RUN LOCALLY!!! You can download it as an .ipynb file and open that with Jupyter notebook/labs

# SQL

In [71]:
#pip install pymysql

In [72]:
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
import pandas as pd

If you get an error saying something like "sqlalchemy not installed", you should add a code cell and write `pip install sqlalchemy`. Then, rerun the above cell (you might also need to install pymysql the same way)

In [73]:
password = 'Warning//2000'
db_name = 'music'

In [74]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', password=password, database=db_name)

In [75]:
connection = create_engine("mysql+pymysql://root:" + password + "@localhost:3306/" + db_name)

In [76]:
songs = pd.read_csv('songs.csv')
albums = pd.read_csv('albums.csv')
artists = pd.read_csv('artists.csv')
playlists = pd.read_csv('playlists.csv')
users = pd.read_csv('users.csv')

In [77]:
table_name = 'songs'
table_name1 = 'albums'
table_name2 = 'artists'
table_name3 = 'playlists'
table_name4 = 'users'

In [78]:
songs.to_sql(name=table_name, con=connection, if_exists='replace', index=False)
albums.to_sql(name=table_name1, con=connection, if_exists='replace', index=False)
artists.to_sql(name=table_name2, con=connection, if_exists='replace', index=False)
playlists.to_sql(name=table_name3, con=connection, if_exists='replace', index=False)
users.to_sql(name=table_name4, con=connection, if_exists='replace', index=False)

10

We use SQL syntax for querying. The triple quotation mark syntax allows us to write commands simiar to how we did in SQL

In [79]:
query1 = """
SELECT * FROM songs
WHERE Genre = 'Pop'
ORDER BY Rating DESC
LIMIT 10
"""

In [80]:
pop_songs = pd.read_sql(query1, con=connection)

In [81]:
pop_songs

,Song_ID,Title,Artist_ID,Album_ID,Playlist_ID,Genre,Rating,Duration
0,13,I Knew You Were Trouble,7.0,3.0,9.0,Pop,10,4:06
1,12,When We Were Young,NaN,NaN,4.0,Pop,10,4:30
2,3,Deja Vu,1.0,1.0,7.0,Pop,10,4:01
3,19,Vampire,1.0,NaN,3.0,Pop,10,3:53
4,41,Love Story,7.0,3.0,5.0,Pop,9,3:55
5,54,Stuck In The Middle,15.0,11.0,NaN,Pop,9,3:16
6,42,Blank Space,7.0,NaN,8.0,Pop,9,3:51
7,36,Watermelon Sugar,8.0,2.0,2.0,Pop,9,2:54
8,57,Spinnin,16.0,12.0,NaN,Pop,9,1:50
9,14,All Too Well,7.0,3.0,5.0,Pop,9,10:21


In [82]:
query2 = """
  SELECT Genre, AVG(Rating)
  FROM songs
  GROUP BY Genre
"""

In [83]:
genre_ratings = pd.read_sql(query2, con=connection)
genre_ratings

,Genre,AVG(Rating)
0,Pop,7.8056
1,Rock,8.0000
2,Indie,6.8462
3,Bollywood,9.0000
4,Alternative Rock,8.0000


We can merge in PyMySQL just like in regular SQL

In [92]:
query3 = """
  SELECT users.Username, users.Country, songs.Title AS Fav_Song, Artist_Country, songs.Rating
  FROM users
  LEFT JOIN songs ON users.Top_Song_ID = songs.Song_ID
  LEFT JOIN artists ON songs.Artist_ID = artists.Artist_ID
  WHERE users.Country <> artists.Artist_Country
  ORDER BY songs.Rating DESC
"""

In [93]:
countries = pd.read_sql(query3, con=connection)
countries

,Username,Country,Fav_Song,Artist_Country,Rating
0,HexRyn,Sweden,Vampire,USA,10
1,KyroM,USA,Watermelon Sugar,UK,9
2,VimJet,Ireland,Happier,USA,7
3,FizzDart,Mexico,My Worst Enemy,USA,7
4,Blink21,USA,Golden,UK,7
5,QuikFox,Canada,Golden,UK,7
6,LunaPix,Canada,Two-Headed Boy,USA,4


In [86]:
query4 = """
SELECT Album_title, Album_release_date 
FROM albums
WHERE STR_TO_DATE(Album_release_date, '%%m/%%d/%%Y') < STR_TO_DATE('02/02/2010', '%%m/%%d/%%Y');
"""

In [89]:
new = pd.read_sql(query4, con=connection)
new

,Album_title,Album_release_date
0,Parachutes,7/10/2000
1,American Idiot,7/25/2004
2,In the Areoplane Over the Sea,2/10/1998


# MONGO

In [112]:
import pymongo
from pymongo import MongoClient
import pandas as pd

In [113]:
client = MongoClient('localhost', 27017)
client
db = client.example

In [114]:
#users

In [115]:
merged_df = pd.merge(playlists, songs, on='Playlist_ID', how='left')
playlist_songs = merged_df.groupby('Playlist_ID')['Title'].apply(list).reset_index()
playlists_df = pd.merge(playlists, playlist_songs, on='Playlist_ID', how='left')
playlists_df['Title'] = playlists_df['Title'].apply(lambda x: x if isinstance(x, list) else [])
playlists_df = playlists_df.rename(columns={'Title': 'Songs_in_playlist'})
print(playlists_df.head(5))

   Playlist_ID  Playlist_name  Creator_ID                    Description  \
0            1   Driving Time           1             Songs to drive to.   
1            2            Gym           2             Songs for the gym.   
2            3   Cooking Time           4        Songs for cooking time.   
3            4          Happy           3  When you have had a GREAT day   
4            5  cleaning time           6                    bops fr fr    

   Popularity                                  Songs_in_playlist  
0        1000  [Happier, Golden , Love to Dream, Yellow, Gold...  
1        2500  [White Horse, High Hopes, Watermelon Sugar, Sh...  
2        1200  [Two-Headed Boy, Vampire, Love Like This, All ...  
3           0  [Enough for You, When We Were Young , Baby for...  
4           1  [Kiss Me More, Adore You, All Too Well, Love S...  


In [116]:
users_dict = users.to_dict('records')
users_dict
songs_dict = songs.to_dict('records')
albums_dict = albums.to_dict('records')
playlists_dict = playlists_df.to_dict('records')
artists_dict = artists.to_dict('records')

In [117]:
users_collection = db.users
albums_collection = db.albums
artists_collection = db.artists
playlists_collection = db.playlists
songs_collection = db.songs

In [118]:
users_collection.insert_many(users_dict)
albums_collection.insert_many(albums_dict)
songs_collection.insert_many(songs_dict)
artists_collection.insert_many(artists_dict)
playlists_collection.insert_many(playlists_dict)

InsertManyResult([ObjectId('665734d4c72e83c5aebaedfc'), ObjectId('665734d4c72e83c5aebaedfd'), ObjectId('665734d4c72e83c5aebaedfe'), ObjectId('665734d4c72e83c5aebaedff'), ObjectId('665734d4c72e83c5aebaee00'), ObjectId('665734d4c72e83c5aebaee01'), ObjectId('665734d4c72e83c5aebaee02'), ObjectId('665734d4c72e83c5aebaee03'), ObjectId('665734d4c72e83c5aebaee04'), ObjectId('665734d4c72e83c5aebaee05')], acknowledged=True)

In [174]:
### queries
print("Sample user document:", users_collection.find_one())

Sample user document: {'_id': ObjectId('665734d4c72e83c5aebaed9c'), 'Creator_ID': 1, 'Username': 'Blink21', 'Country': 'USA', 'Top_Song_ID': 9.0, 'Top_Album_ID': 2.0, 'Top_Artist_ID': 8.0, 'Best_friend_ID': 3.0}


In [184]:
pipeline = [
    {
        "$group": {
            "_id": "$Genre",
            "average_rating": {"$avg": "$Rating"}
        }
    }
]
  
    
results = songs_collection.aggregate(pipeline)
df = pd.DataFrame(list(results))
df

,_id,average_rating
0,Bollywood,9.000000
1,Pop,7.805556
2,Rock,8.000000
3,Indie,6.846154
4,Alternative Rock,8.000000


In [111]:
### dont run until the end

users_collection.delete_many({})
songs_collection.delete_many({})
albums_collection.delete_many({})
artists_collection.delete_many({})
playlists_collection.delete_many({})
db.users_collection.drop()
db.songs_collection.drop()
db.albums_collection.drop()
db.artists_collection.drop()
db.playlists_collection.drop()